# Homework 6

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner
import numpy as np
import os
import tempfile
import pdb


BLOCKS_DOMAIN = """(define (domain prodigy-bw)
  (:requirements :strips)
  (:predicates (on ?x ?y)
	       (on-table ?x)
	       (clear ?x)
	       (arm-empty)
	       (holding ?x)
	       )
  (:action pick-up
	     :parameters (?ob1)
	     :precondition (and (clear ?ob1) (on-table ?ob1) (arm-empty))
	     :effect
	     (and (not (on-table ?ob1))
		   (not (clear ?ob1))
		   (not (arm-empty))
		   (holding ?ob1)))
  (:action put-down
	     :parameters (?ob)
	     :precondition (holding ?ob)
	     :effect
	     (and (not (holding ?ob))
		   (clear ?ob)
		   (arm-empty)
		   (on-table ?ob)))
  (:action stack
	     :parameters (?sob ?sunderob)
	     :precondition (and (holding ?sob) (clear ?sunderob))
	     :effect
	     (and (not (holding ?sob))
		   (not (clear ?sunderob))
		   (clear ?sob)
		   (arm-empty)
		   (on ?sob ?sunderob)))
  (:action unstack
	     :parameters (?sob ?sunderob)
	     :precondition (and (on ?sob ?sunderob) (clear ?sob) (arm-empty))
	     :effect
	     (and (holding ?sob)
		   (clear ?sunderob)
		   (not (clear ?sob))
		   (not (arm-empty))
		   (not (on ?sob ?sunderob)))))
"""

BLOCKS_PROBLEM_1 = """(define (problem bw-simple)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (clear a) (arm-empty) (on a b) (on-table b))
  (:goal (and (on-table a) (clear b))))
"""

BLOCKS_PROBLEM_2 = """(define (problem bw-sussman)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (on-table a) (on-table b) (on c a)
		(clear b) (clear c) (arm-empty))
  (:goal (and (on a b) (on b c))))
"""

BLOCKS_PROBLEM_3 = """(define (problem bw-large-a)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9)
  (:init (arm-empty)
	 (on 3 2)
	 (on 2 1)
	 (on-table 1)
	 (on 5 4)
	 (on-table 4)
	 (on 9 8)
	 (on 8 7)
	 (on 7 6)
	 (on-table 6)
	 (clear 3)
	 (clear 5)
	 (clear 9))
  (:goal (and
	  (on 1 5)
	  (on-table 5)
	  (on 8 9)
	  (on 9 4)
	  (on-table 4)
	  (on 2 3)
	  (on 3 7)
	  (on 7 6)
	  (on-table 6)
	  (clear 1)
	  (clear 8)
	  (clear 2)
	  )))
"""

BLOCKS_PROBLEM_4 = """(define (problem bw-large-b)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9 10 11)
  (:init (arm-empty)
	 (on 3 2)
	 (on 2 1)
	 (on-table 1)
	 (on 11 10)
	 (on 10 5)
	 (on 5 4)
	 (on-table 4)
	 (on 9 8)
	 (on 8 7)
	 (on 7 6)
	 (on-table 6)
	 (clear 3)
	 (clear 11)
	 (clear 9))
  (:goal (and
	  (on 1 5)
	  (on 5 10)
	  (on-table 10)
	  (on 8 9)
	  (on 9 4)
	  (on-table 4)
	  (on 2 3)
	  (on 3 11)
	  (on 11 7)
	  (on 7 6)
	  (on-table 6)
	  (clear 1)
	  (clear 8)
	  (clear 2)
	  )))
"""

def get_task_definition_str(domain_pddl_str, problem_pddl_str):
  """Get pyperplan task definition from PDDL domain and problem

  This function is a lightweight wrapper around pyperplan.

  Args:
    domain_pddl_str: A str, the contents of a domain.pddl file.
    problem_pddl_str: A str, the contents of a problem.pddl file.

  Returns:
    task: a structure defining the problem
  """
  # Parsing the PDDDL
  domain_file = tempfile.NamedTemporaryFile(delete=False)
  problem_file = tempfile.NamedTemporaryFile(delete=False)
  with open(domain_file.name, 'w') as f:
    f.write(domain_pddl_str)
  with open(problem_file.name, 'w') as f:
    f.write(problem_pddl_str)
  parser = Parser(domain_file.name, problem_file.name)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)
  os.remove(domain_file.name)
  os.remove(problem_file.name)

  # Ground the PDDL
  task = grounding.ground(problem)

  return task

def get_task_definition(domain_pddl, problem_pddl):
  """Get pyperplan task definition from PDDL domain and problem

  This function is a lightweight wrapper around pyperplan.

  Args:
    domain_pddl_str: A str, the name of a domain.pddl file.
    problem_pddl_str: A str, the name of a problem.pddl file.

  Returns:
    task: a structure defining the problem
  """
  # Parsing the PDDDL
  parser = Parser(domain_pddl, problem_pddl)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)

  # Ground the PDDL
  task = grounding.ground(problem)

  return task

def run_on_task_and_state(fn, domain_pddl_str, problem_pddl_str, state=None):
    task = get_task_definition_str(domain_pddl_str, problem_pddl_str)
    return fn(task, state)




## Problems

### Construct Reduced Planning Graph (RPG)
Given a Pyperplan Task instance and an optional state, return (F_t, A_t), two lists as defined in the RPG pseudo-code from lecture.

For reference, our solution is **13** lines of code.

In [ ]:
def construct_rpg(task, state=None):
  """Constructs a Relaxed Planning Graph (RPG) for domain, problem, and state.

  Args:
    task: a pyperplan Task instance
    state: a set of facts

  Returns:
    rpg: F_sets (list of fact sets), A_sets (list of operator sets)
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def construct_rpg_test1():
  task = get_task_definition_str(BLOCKS_DOMAIN, BLOCKS_PROBLEM_1)
  expected_F_sets = [{'(clear a)', '(on a b)', '(on-table b)', '(arm-empty)'}, {'(on-table b)', '(arm-empty)', '(clear a)', '(on a b)', '(holding a)', '(clear b)'}, {'(arm-empty)', '(on-table a)', '(holding a)', '(holding b)', '(on-table b)', '(on a a)', '(clear a)', '(on a b)', '(clear b)'}]
  F_sets, A_sets = construct_rpg(task)
  assert len(F_sets) == len(expected_F_sets)
  for F_set in F_sets:
      assert F_set in expected_F_sets
  expected_A_set_strs = ['{<Op (unstack a b)>}', '{<Op (unstack a b)>, <Op (pick-up b)>, <Op (stack a a)>, <Op (stack a b)>, <Op (put-down a)>}']
  assert len(A_sets) == len(expected_A_set_strs)
  for A_set in A_sets:
      assert str(A_set) in expected_A_set_strs

construct_rpg_test1()
print('Tests passed.')

### h_add
Given a Pyperplan Taks instance and a state (set of facts or None), return h_add(state).

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [ ]:
def h_add(task, state=None):
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

assert h_add(Task bw-simple
  Vars:  (clear b), (on c a), (on-table a), (on b b), (on-table b), (clear c), (on a a), (on c c), (holding c), (on b c), (arm-empty), (holding a), (on c b), (holding b), (on b a), (on a c), (clear a), (on a b), (on-table c)
  Init:  frozenset({"(clear a)", "(on a b)", "(on-table b)", "(arm-empty)"})
  Goals: frozenset({"(on-table a)", "(clear b)"})
  Ops:   <Op (pick-up a)>
<Op (pick-up b)>
<Op (pick-up c)>
<Op (put-down a)>
<Op (put-down b)>
<Op (put-down c)>
<Op (stack a a)>
<Op (stack a b)>
<Op (stack a c)>
<Op (stack b a)>
<Op (stack b b)>
<Op (stack b c)>
<Op (stack c a)>
<Op (stack c b)>
<Op (stack c c)>
<Op (unstack a a)>
<Op (unstack a b)>
<Op (unstack a c)>
<Op (unstack b a)>
<Op (unstack b b)>
<Op (unstack b c)>
<Op (unstack c a)>
<Op (unstack c b)>
<Op (unstack c c)>) == 3

assert h_add(Task bw-sussman
  Vars:  (clear b), (on c a), (on-table a), (on b b), (on-table b), (clear c), (on a a), (on c c), (holding c), (on b c), (arm-empty), (holding a), (on c b), (holding b), (on b a), (on a c), (clear a), (on a b), (on-table c)
  Init:  frozenset({"(arm-empty)", "(on c a)", "(on-table a)", "(on-table b)", "(clear c)", "(clear b)"})
  Goals: frozenset({"(on a b)", "(on b c)"})
  Ops:   <Op (pick-up a)>
<Op (pick-up b)>
<Op (pick-up c)>
<Op (put-down a)>
<Op (put-down b)>
<Op (put-down c)>
<Op (stack a a)>
<Op (stack a b)>
<Op (stack a c)>
<Op (stack b a)>
<Op (stack b b)>
<Op (stack b c)>
<Op (stack c a)>
<Op (stack c b)>
<Op (stack c c)>
<Op (unstack a a)>
<Op (unstack a b)>
<Op (unstack a c)>
<Op (unstack b a)>
<Op (unstack b b)>
<Op (unstack b c)>
<Op (unstack c a)>
<Op (unstack c b)>
<Op (unstack c c)>) == 5

assert h_add(Task bw-large-a
  Vars:  (on 8 7), (on 6 6), (on 3 7), (on 8 1), (on 5 2), (on 2 8), (on 7 3), (on 7 1), (on 2 7), (on 9 7), (holding 4), (on 5 7), (on 6 5), (clear 6), (on-table 8), (on 5 4), (on 4 1), (on-table 4), (on 2 3), (on 5 3), (clear 5), (clear 8), (on 3 4), (on 4 8), (on 2 2), (on 5 1), (on 8 3), (on 3 6), (on 6 4), (on 1 8), (on 4 5), (on 6 8), (on 1 4), (holding 2), (holding 5), (on 2 5), (on 5 6), (on 4 6), (on 9 1), (on 2 6), (on 7 9), (clear 7), (on 7 6), (on-table 1), (clear 2), (on 3 3), (on 2 1), (on 8 2), (on 3 9), (clear 3), (on 1 1), (on 9 8), (on 8 6), (on 4 7), (on 5 9), (on 7 7), (on 7 8), (on 9 6), (on-table 7), (on 9 9), (on-table 9), (holding 3), (on 3 1), (on 1 9), (on 8 4), (on 9 3), (clear 4), (holding 9), (on 2 4), (holding 6), (on 3 2), (on 8 8), (on 8 5), (on-table 2), (holding 8), (on 1 7), (on 4 3), (on 4 9), (on 3 5), (on 1 6), (on 5 8), (on 9 2), (on-table 5), (on 6 7), (on 7 2), (on 1 3), (on 2 9), (on 4 2), (clear 1), (on-table 6), (on 8 9), (on 9 4), (on 6 2), (on 6 1), (arm-empty), (holding 7), (on 1 5), (on 5 5), (on 7 4), (on 7 5), (on-table 3), (clear 9), (on 4 4), (on 9 5), (holding 1), (on 3 8), (on 6 9), (on 1 2), (on 6 3)
  Init:  frozenset({"(on 8 7)", "(on-table 4)", "(arm-empty)", "(on 7 6)", "(on 3 2)", "(clear 9)", "(clear 5)", "(on 2 1)", "(on-table 1)", "(on 9 8)", "(clear 3)", "(on 5 4)", "(on-table 6)"})
  Goals: frozenset({"(on-table 4)", "(on 3 7)", "(on 2 3)", "(on-table 6)", "(on 1 5)", "(on 7 6)", "(clear 8)", "(clear 2)", "(on-table 5)", "(clear 1)", "(on 9 4)", "(on 8 9)"})
  Ops:   <Op (pick-up 5)>
<Op (pick-up 3)>
<Op (pick-up 2)>
<Op (pick-up 7)>
<Op (pick-up 4)>
<Op (pick-up 1)>
<Op (pick-up 8)>
<Op (pick-up 9)>
<Op (pick-up 6)>
<Op (put-down 5)>
<Op (put-down 3)>
<Op (put-down 2)>
<Op (put-down 7)>
<Op (put-down 4)>
<Op (put-down 1)>
<Op (put-down 8)>
<Op (put-down 9)>
<Op (put-down 6)>
<Op (stack 5 5)>
<Op (stack 5 3)>
<Op (stack 5 2)>
<Op (stack 5 7)>
<Op (stack 5 4)>
<Op (stack 5 1)>
<Op (stack 5 8)>
<Op (stack 5 9)>
<Op (stack 5 6)>
<Op (stack 3 5)>
<Op (stack 3 3)>
<Op (stack 3 2)>
<Op (stack 3 7)>
<Op (stack 3 4)>
<Op (stack 3 1)>
<Op (stack 3 8)>
<Op (stack 3 9)>
<Op (stack 3 6)>
<Op (stack 2 5)>
<Op (stack 2 3)>
<Op (stack 2 2)>
<Op (stack 2 7)>
<Op (stack 2 4)>
<Op (stack 2 1)>
<Op (stack 2 8)>
<Op (stack 2 9)>
<Op (stack 2 6)>
<Op (stack 7 5)>
<Op (stack 7 3)>
<Op (stack 7 2)>
<Op (stack 7 7)>
<Op (stack 7 4)>
<Op (stack 7 1)>
<Op (stack 7 8)>
<Op (stack 7 9)>
<Op (stack 7 6)>
<Op (stack 4 5)>
<Op (stack 4 3)>
<Op (stack 4 2)>
<Op (stack 4 7)>
<Op (stack 4 4)>
<Op (stack 4 1)>
<Op (stack 4 8)>
<Op (stack 4 9)>
<Op (stack 4 6)>
<Op (stack 1 5)>
<Op (stack 1 3)>
<Op (stack 1 2)>
<Op (stack 1 7)>
<Op (stack 1 4)>
<Op (stack 1 1)>
<Op (stack 1 8)>
<Op (stack 1 9)>
<Op (stack 1 6)>
<Op (stack 8 5)>
<Op (stack 8 3)>
<Op (stack 8 2)>
<Op (stack 8 7)>
<Op (stack 8 4)>
<Op (stack 8 1)>
<Op (stack 8 8)>
<Op (stack 8 9)>
<Op (stack 8 6)>
<Op (stack 9 5)>
<Op (stack 9 3)>
<Op (stack 9 2)>
<Op (stack 9 7)>
<Op (stack 9 4)>
<Op (stack 9 1)>
<Op (stack 9 8)>
<Op (stack 9 9)>
<Op (stack 9 6)>
<Op (stack 6 5)>
<Op (stack 6 3)>
<Op (stack 6 2)>
<Op (stack 6 7)>
<Op (stack 6 4)>
<Op (stack 6 1)>
<Op (stack 6 8)>
<Op (stack 6 9)>
<Op (stack 6 6)>
<Op (unstack 5 5)>
<Op (unstack 5 3)>
<Op (unstack 5 2)>
<Op (unstack 5 7)>
<Op (unstack 5 4)>
<Op (unstack 5 1)>
<Op (unstack 5 8)>
<Op (unstack 5 9)>
<Op (unstack 5 6)>
<Op (unstack 3 5)>
<Op (unstack 3 3)>
<Op (unstack 3 2)>
<Op (unstack 3 7)>
<Op (unstack 3 4)>
<Op (unstack 3 1)>
<Op (unstack 3 8)>
<Op (unstack 3 9)>
<Op (unstack 3 6)>
<Op (unstack 2 5)>
<Op (unstack 2 3)>
<Op (unstack 2 2)>
<Op (unstack 2 7)>
<Op (unstack 2 4)>
<Op (unstack 2 1)>
<Op (unstack 2 8)>
<Op (unstack 2 9)>
<Op (unstack 2 6)>
<Op (unstack 7 5)>
<Op (unstack 7 3)>
<Op (unstack 7 2)>
<Op (unstack 7 7)>
<Op (unstack 7 4)>
<Op (unstack 7 1)>
<Op (unstack 7 8)>
<Op (unstack 7 9)>
<Op (unstack 7 6)>
<Op (unstack 4 5)>
<Op (unstack 4 3)>
<Op (unstack 4 2)>
<Op (unstack 4 7)>
<Op (unstack 4 4)>
<Op (unstack 4 1)>
<Op (unstack 4 8)>
<Op (unstack 4 9)>
<Op (unstack 4 6)>
<Op (unstack 1 5)>
<Op (unstack 1 3)>
<Op (unstack 1 2)>
<Op (unstack 1 7)>
<Op (unstack 1 4)>
<Op (unstack 1 1)>
<Op (unstack 1 8)>
<Op (unstack 1 9)>
<Op (unstack 1 6)>
<Op (unstack 8 5)>
<Op (unstack 8 3)>
<Op (unstack 8 2)>
<Op (unstack 8 7)>
<Op (unstack 8 4)>
<Op (unstack 8 1)>
<Op (unstack 8 8)>
<Op (unstack 8 9)>
<Op (unstack 8 6)>
<Op (unstack 9 5)>
<Op (unstack 9 3)>
<Op (unstack 9 2)>
<Op (unstack 9 7)>
<Op (unstack 9 4)>
<Op (unstack 9 1)>
<Op (unstack 9 8)>
<Op (unstack 9 9)>
<Op (unstack 9 6)>
<Op (unstack 6 5)>
<Op (unstack 6 3)>
<Op (unstack 6 2)>
<Op (unstack 6 7)>
<Op (unstack 6 4)>
<Op (unstack 6 1)>
<Op (unstack 6 8)>
<Op (unstack 6 9)>
<Op (unstack 6 6)>) == 21

assert h_add(Task bw-large-b
  Vars:  (on 8 7), (on 11 4), (on 3 7), (on-table 11), (on 8 1), (on 7 1), (on 10 10), (holding 4), (on 5 7), (clear 6), (on 11 1), (on 2 11), (on-table 8), (on 5 4), (on 5 11), (on 4 1), (on-table 4), (on 7 11), (on 2 3), (on 10 8), (clear 8), (on 3 4), (on 4 8), (on 2 2), (on 5 1), (on 8 3), (on 6 4), (on 1 8), (on 4 10), (on 4 5), (on 6 8), (on 2 5), (on 5 6), (on 9 1), (on 7 6), (on 11 11), (on 11 10), (on 3 9), (holding 10), (on 10 4), (clear 3), (on 1 1), (on 5 10), (on 4 7), (on 9 6), (on-table 7), (on 8 10), (on-table 9), (holding 3), (on 3 1), (on 3 11), (on 2 4), (on-table 10), (holding 6), (on 3 2), (on 8 5), (on 1 10), (holding 8), (on 10 6), (on 9 2), (on-table 5), (on 7 2), (on 3 10), (on 10 7), (on 8 9), (on 6 2), (arm-empty), (on 1 5), (on 4 4), (clear 9), (on 9 5), (on 6 9), (on 3 8), (on 6 10), (holding 11), (on 6 6), (on 5 2), (on 2 8), (on 7 3), (on 11 7), (on 2 7), (on 4 11), (on 9 7), (on 6 5), (on 11 5), (on 11 9), (on 10 2), (on 5 3), (clear 5), (on 3 6), (clear 10), (on 8 11), (on 11 6), (on 10 3), (on 1 4), (holding 2), (holding 5), (on 6 11), (on 4 6), (on 10 11), (on 2 6), (on 10 9), (on 7 9), (clear 7), (on-table 1), (clear 2), (on 3 3), (on 2 1), (on 11 8), (on 9 11), (on 8 2), (on 9 8), (on 10 5), (on 8 6), (on 5 9), (on 7 7), (on 7 8), (on 9 9), (on 8 4), (on 1 9), (on 9 3), (clear 11), (on 7 10), (on 11 2), (clear 4), (holding 9), (on 2 10), (on 8 8), (on-table 2), (on 1 7), (on 4 3), (on 4 9), (on 3 5), (on 10 1), (on 1 6), (on 5 8), (on 6 7), (on 1 3), (on 2 9), (on 9 10), (on 4 2), (clear 1), (on 11 3), (on-table 6), (on 9 4), (on 6 1), (holding 7), (on 5 5), (on 7 4), (on 7 5), (on-table 3), (holding 1), (on 1 2), (on 6 3), (on 1 11)
  Init:  frozenset({"(on 8 7)", "(on-table 4)", "(arm-empty)", "(on 7 6)", "(on 3 2)", "(clear 9)", "(on 2 1)", "(on-table 1)", "(on 11 10)", "(on 9 8)", "(clear 11)", "(on 10 5)", "(clear 3)", "(on 5 4)", "(on-table 6)"})
  Goals: frozenset({"(on-table 4)", "(on 3 11)", "(on 2 3)", "(on 11 7)", "(on 1 5)", "(on 7 6)", "(on-table 10)", "(on-table 6)", "(clear 8)", "(clear 2)", "(on 5 10)", "(clear 1)", "(on 9 4)", "(on 8 9)"})
  Ops:   <Op (pick-up 10)>
<Op (pick-up 5)>
<Op (pick-up 3)>
<Op (pick-up 2)>
<Op (pick-up 7)>
<Op (pick-up 4)>
<Op (pick-up 1)>
<Op (pick-up 8)>
<Op (pick-up 9)>
<Op (pick-up 11)>
<Op (pick-up 6)>
<Op (put-down 10)>
<Op (put-down 5)>
<Op (put-down 3)>
<Op (put-down 2)>
<Op (put-down 7)>
<Op (put-down 4)>
<Op (put-down 1)>
<Op (put-down 8)>
<Op (put-down 9)>
<Op (put-down 11)>
<Op (put-down 6)>
<Op (stack 10 10)>
<Op (stack 10 5)>
<Op (stack 10 3)>
<Op (stack 10 2)>
<Op (stack 10 7)>
<Op (stack 10 4)>
<Op (stack 10 1)>
<Op (stack 10 8)>
<Op (stack 10 9)>
<Op (stack 10 11)>
<Op (stack 10 6)>
<Op (stack 5 10)>
<Op (stack 5 5)>
<Op (stack 5 3)>
<Op (stack 5 2)>
<Op (stack 5 7)>
<Op (stack 5 4)>
<Op (stack 5 1)>
<Op (stack 5 8)>
<Op (stack 5 9)>
<Op (stack 5 11)>
<Op (stack 5 6)>
<Op (stack 3 10)>
<Op (stack 3 5)>
<Op (stack 3 3)>
<Op (stack 3 2)>
<Op (stack 3 7)>
<Op (stack 3 4)>
<Op (stack 3 1)>
<Op (stack 3 8)>
<Op (stack 3 9)>
<Op (stack 3 11)>
<Op (stack 3 6)>
<Op (stack 2 10)>
<Op (stack 2 5)>
<Op (stack 2 3)>
<Op (stack 2 2)>
<Op (stack 2 7)>
<Op (stack 2 4)>
<Op (stack 2 1)>
<Op (stack 2 8)>
<Op (stack 2 9)>
<Op (stack 2 11)>
<Op (stack 2 6)>
<Op (stack 7 10)>
<Op (stack 7 5)>
<Op (stack 7 3)>
<Op (stack 7 2)>
<Op (stack 7 7)>
<Op (stack 7 4)>
<Op (stack 7 1)>
<Op (stack 7 8)>
<Op (stack 7 9)>
<Op (stack 7 11)>
<Op (stack 7 6)>
<Op (stack 4 10)>
<Op (stack 4 5)>
<Op (stack 4 3)>
<Op (stack 4 2)>
<Op (stack 4 7)>
<Op (stack 4 4)>
<Op (stack 4 1)>
<Op (stack 4 8)>
<Op (stack 4 9)>
<Op (stack 4 11)>
<Op (stack 4 6)>
<Op (stack 1 10)>
<Op (stack 1 5)>
<Op (stack 1 3)>
<Op (stack 1 2)>
<Op (stack 1 7)>
<Op (stack 1 4)>
<Op (stack 1 1)>
<Op (stack 1 8)>
<Op (stack 1 9)>
<Op (stack 1 11)>
<Op (stack 1 6)>
<Op (stack 8 10)>
<Op (stack 8 5)>
<Op (stack 8 3)>
<Op (stack 8 2)>
<Op (stack 8 7)>
<Op (stack 8 4)>
<Op (stack 8 1)>
<Op (stack 8 8)>
<Op (stack 8 9)>
<Op (stack 8 11)>
<Op (stack 8 6)>
<Op (stack 9 10)>
<Op (stack 9 5)>
<Op (stack 9 3)>
<Op (stack 9 2)>
<Op (stack 9 7)>
<Op (stack 9 4)>
<Op (stack 9 1)>
<Op (stack 9 8)>
<Op (stack 9 9)>
<Op (stack 9 11)>
<Op (stack 9 6)>
<Op (stack 11 10)>
<Op (stack 11 5)>
<Op (stack 11 3)>
<Op (stack 11 2)>
<Op (stack 11 7)>
<Op (stack 11 4)>
<Op (stack 11 1)>
<Op (stack 11 8)>
<Op (stack 11 9)>
<Op (stack 11 11)>
<Op (stack 11 6)>
<Op (stack 6 10)>
<Op (stack 6 5)>
<Op (stack 6 3)>
<Op (stack 6 2)>
<Op (stack 6 7)>
<Op (stack 6 4)>
<Op (stack 6 1)>
<Op (stack 6 8)>
<Op (stack 6 9)>
<Op (stack 6 11)>
<Op (stack 6 6)>
<Op (unstack 10 10)>
<Op (unstack 10 5)>
<Op (unstack 10 3)>
<Op (unstack 10 2)>
<Op (unstack 10 7)>
<Op (unstack 10 4)>
<Op (unstack 10 1)>
<Op (unstack 10 8)>
<Op (unstack 10 9)>
<Op (unstack 10 11)>
<Op (unstack 10 6)>
<Op (unstack 5 10)>
<Op (unstack 5 5)>
<Op (unstack 5 3)>
<Op (unstack 5 2)>
<Op (unstack 5 7)>
<Op (unstack 5 4)>
<Op (unstack 5 1)>
<Op (unstack 5 8)>
<Op (unstack 5 9)>
<Op (unstack 5 11)>
<Op (unstack 5 6)>
<Op (unstack 3 10)>
<Op (unstack 3 5)>
<Op (unstack 3 3)>
<Op (unstack 3 2)>
<Op (unstack 3 7)>
<Op (unstack 3 4)>
<Op (unstack 3 1)>
<Op (unstack 3 8)>
<Op (unstack 3 9)>
<Op (unstack 3 11)>
<Op (unstack 3 6)>
<Op (unstack 2 10)>
<Op (unstack 2 5)>
<Op (unstack 2 3)>
<Op (unstack 2 2)>
<Op (unstack 2 7)>
<Op (unstack 2 4)>
<Op (unstack 2 1)>
<Op (unstack 2 8)>
<Op (unstack 2 9)>
<Op (unstack 2 11)>
<Op (unstack 2 6)>
<Op (unstack 7 10)>
<Op (unstack 7 5)>
<Op (unstack 7 3)>
<Op (unstack 7 2)>
<Op (unstack 7 7)>
<Op (unstack 7 4)>
<Op (unstack 7 1)>
<Op (unstack 7 8)>
<Op (unstack 7 9)>
<Op (unstack 7 11)>
<Op (unstack 7 6)>
<Op (unstack 4 10)>
<Op (unstack 4 5)>
<Op (unstack 4 3)>
<Op (unstack 4 2)>
<Op (unstack 4 7)>
<Op (unstack 4 4)>
<Op (unstack 4 1)>
<Op (unstack 4 8)>
<Op (unstack 4 9)>
<Op (unstack 4 11)>
<Op (unstack 4 6)>
<Op (unstack 1 10)>
<Op (unstack 1 5)>
<Op (unstack 1 3)>
<Op (unstack 1 2)>
<Op (unstack 1 7)>
<Op (unstack 1 4)>
<Op (unstack 1 1)>
<Op (unstack 1 8)>
<Op (unstack 1 9)>
<Op (unstack 1 11)>
<Op (unstack 1 6)>
<Op (unstack 8 10)>
<Op (unstack 8 5)>
<Op (unstack 8 3)>
<Op (unstack 8 2)>
<Op (unstack 8 7)>
<Op (unstack 8 4)>
<Op (unstack 8 1)>
<Op (unstack 8 8)>
<Op (unstack 8 9)>
<Op (unstack 8 11)>
<Op (unstack 8 6)>
<Op (unstack 9 10)>
<Op (unstack 9 5)>
<Op (unstack 9 3)>
<Op (unstack 9 2)>
<Op (unstack 9 7)>
<Op (unstack 9 4)>
<Op (unstack 9 1)>
<Op (unstack 9 8)>
<Op (unstack 9 9)>
<Op (unstack 9 11)>
<Op (unstack 9 6)>
<Op (unstack 11 10)>
<Op (unstack 11 5)>
<Op (unstack 11 3)>
<Op (unstack 11 2)>
<Op (unstack 11 7)>
<Op (unstack 11 4)>
<Op (unstack 11 1)>
<Op (unstack 11 8)>
<Op (unstack 11 9)>
<Op (unstack 11 11)>
<Op (unstack 11 6)>
<Op (unstack 6 10)>
<Op (unstack 6 5)>
<Op (unstack 6 3)>
<Op (unstack 6 2)>
<Op (unstack 6 7)>
<Op (unstack 6 4)>
<Op (unstack 6 1)>
<Op (unstack 6 8)>
<Op (unstack 6 9)>
<Op (unstack 6 11)>
<Op (unstack 6 6)>) == 30
print('Tests passed.')

### h_ff
Given a Pyperplan Taks instance and a state (set of facts or None), return h_ff(state).

For reference, our solution is **18** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [ ]:
def h_ff(task, state=None):
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

assert h_ff(Task bw-simple
  Vars:  (clear b), (on c a), (on-table a), (on b b), (on-table b), (clear c), (on a a), (on c c), (holding c), (on b c), (arm-empty), (holding a), (on c b), (holding b), (on b a), (on a c), (clear a), (on a b), (on-table c)
  Init:  frozenset({"(clear a)", "(on a b)", "(on-table b)", "(arm-empty)"})
  Goals: frozenset({"(on-table a)", "(clear b)"})
  Ops:   <Op (pick-up a)>
<Op (pick-up b)>
<Op (pick-up c)>
<Op (put-down a)>
<Op (put-down b)>
<Op (put-down c)>
<Op (stack a a)>
<Op (stack a b)>
<Op (stack a c)>
<Op (stack b a)>
<Op (stack b b)>
<Op (stack b c)>
<Op (stack c a)>
<Op (stack c b)>
<Op (stack c c)>
<Op (unstack a a)>
<Op (unstack a b)>
<Op (unstack a c)>
<Op (unstack b a)>
<Op (unstack b b)>
<Op (unstack b c)>
<Op (unstack c a)>
<Op (unstack c b)>
<Op (unstack c c)>) == 2

assert h_ff(Task bw-sussman
  Vars:  (clear b), (on c a), (on-table a), (on b b), (on-table b), (clear c), (on a a), (on c c), (holding c), (on b c), (arm-empty), (holding a), (on c b), (holding b), (on b a), (on a c), (clear a), (on a b), (on-table c)
  Init:  frozenset({"(arm-empty)", "(on c a)", "(on-table a)", "(on-table b)", "(clear c)", "(clear b)"})
  Goals: frozenset({"(on a b)", "(on b c)"})
  Ops:   <Op (pick-up a)>
<Op (pick-up b)>
<Op (pick-up c)>
<Op (put-down a)>
<Op (put-down b)>
<Op (put-down c)>
<Op (stack a a)>
<Op (stack a b)>
<Op (stack a c)>
<Op (stack b a)>
<Op (stack b b)>
<Op (stack b c)>
<Op (stack c a)>
<Op (stack c b)>
<Op (stack c c)>
<Op (unstack a a)>
<Op (unstack a b)>
<Op (unstack a c)>
<Op (unstack b a)>
<Op (unstack b b)>
<Op (unstack b c)>
<Op (unstack c a)>
<Op (unstack c b)>
<Op (unstack c c)>) == 5

assert h_ff(Task bw-large-a
  Vars:  (on 8 7), (on 6 6), (on 3 7), (on 8 1), (on 5 2), (on 2 8), (on 7 3), (on 7 1), (on 2 7), (on 9 7), (holding 4), (on 5 7), (on 6 5), (clear 6), (on-table 8), (on 5 4), (on 4 1), (on-table 4), (on 2 3), (on 5 3), (clear 5), (clear 8), (on 3 4), (on 4 8), (on 2 2), (on 5 1), (on 8 3), (on 3 6), (on 6 4), (on 1 8), (on 4 5), (on 6 8), (on 1 4), (holding 2), (holding 5), (on 2 5), (on 5 6), (on 4 6), (on 9 1), (on 2 6), (on 7 9), (clear 7), (on 7 6), (on-table 1), (clear 2), (on 3 3), (on 2 1), (on 8 2), (on 3 9), (clear 3), (on 1 1), (on 9 8), (on 8 6), (on 4 7), (on 5 9), (on 7 7), (on 7 8), (on 9 6), (on-table 7), (on 9 9), (on-table 9), (holding 3), (on 3 1), (on 1 9), (on 8 4), (on 9 3), (clear 4), (holding 9), (on 2 4), (holding 6), (on 3 2), (on 8 8), (on 8 5), (on-table 2), (holding 8), (on 1 7), (on 4 3), (on 4 9), (on 3 5), (on 1 6), (on 5 8), (on 9 2), (on-table 5), (on 6 7), (on 7 2), (on 1 3), (on 2 9), (on 4 2), (clear 1), (on-table 6), (on 8 9), (on 9 4), (on 6 2), (on 6 1), (arm-empty), (holding 7), (on 1 5), (on 5 5), (on 7 4), (on 7 5), (on-table 3), (clear 9), (on 4 4), (on 9 5), (holding 1), (on 3 8), (on 6 9), (on 1 2), (on 6 3)
  Init:  frozenset({"(on 8 7)", "(on-table 4)", "(arm-empty)", "(on 7 6)", "(on 3 2)", "(clear 9)", "(clear 5)", "(on 2 1)", "(on-table 1)", "(on 9 8)", "(clear 3)", "(on 5 4)", "(on-table 6)"})
  Goals: frozenset({"(on-table 4)", "(on 3 7)", "(on 2 3)", "(on-table 6)", "(on 1 5)", "(on 7 6)", "(clear 8)", "(clear 2)", "(on-table 5)", "(clear 1)", "(on 9 4)", "(on 8 9)"})
  Ops:   <Op (pick-up 5)>
<Op (pick-up 3)>
<Op (pick-up 2)>
<Op (pick-up 7)>
<Op (pick-up 4)>
<Op (pick-up 1)>
<Op (pick-up 8)>
<Op (pick-up 9)>
<Op (pick-up 6)>
<Op (put-down 5)>
<Op (put-down 3)>
<Op (put-down 2)>
<Op (put-down 7)>
<Op (put-down 4)>
<Op (put-down 1)>
<Op (put-down 8)>
<Op (put-down 9)>
<Op (put-down 6)>
<Op (stack 5 5)>
<Op (stack 5 3)>
<Op (stack 5 2)>
<Op (stack 5 7)>
<Op (stack 5 4)>
<Op (stack 5 1)>
<Op (stack 5 8)>
<Op (stack 5 9)>
<Op (stack 5 6)>
<Op (stack 3 5)>
<Op (stack 3 3)>
<Op (stack 3 2)>
<Op (stack 3 7)>
<Op (stack 3 4)>
<Op (stack 3 1)>
<Op (stack 3 8)>
<Op (stack 3 9)>
<Op (stack 3 6)>
<Op (stack 2 5)>
<Op (stack 2 3)>
<Op (stack 2 2)>
<Op (stack 2 7)>
<Op (stack 2 4)>
<Op (stack 2 1)>
<Op (stack 2 8)>
<Op (stack 2 9)>
<Op (stack 2 6)>
<Op (stack 7 5)>
<Op (stack 7 3)>
<Op (stack 7 2)>
<Op (stack 7 7)>
<Op (stack 7 4)>
<Op (stack 7 1)>
<Op (stack 7 8)>
<Op (stack 7 9)>
<Op (stack 7 6)>
<Op (stack 4 5)>
<Op (stack 4 3)>
<Op (stack 4 2)>
<Op (stack 4 7)>
<Op (stack 4 4)>
<Op (stack 4 1)>
<Op (stack 4 8)>
<Op (stack 4 9)>
<Op (stack 4 6)>
<Op (stack 1 5)>
<Op (stack 1 3)>
<Op (stack 1 2)>
<Op (stack 1 7)>
<Op (stack 1 4)>
<Op (stack 1 1)>
<Op (stack 1 8)>
<Op (stack 1 9)>
<Op (stack 1 6)>
<Op (stack 8 5)>
<Op (stack 8 3)>
<Op (stack 8 2)>
<Op (stack 8 7)>
<Op (stack 8 4)>
<Op (stack 8 1)>
<Op (stack 8 8)>
<Op (stack 8 9)>
<Op (stack 8 6)>
<Op (stack 9 5)>
<Op (stack 9 3)>
<Op (stack 9 2)>
<Op (stack 9 7)>
<Op (stack 9 4)>
<Op (stack 9 1)>
<Op (stack 9 8)>
<Op (stack 9 9)>
<Op (stack 9 6)>
<Op (stack 6 5)>
<Op (stack 6 3)>
<Op (stack 6 2)>
<Op (stack 6 7)>
<Op (stack 6 4)>
<Op (stack 6 1)>
<Op (stack 6 8)>
<Op (stack 6 9)>
<Op (stack 6 6)>
<Op (unstack 5 5)>
<Op (unstack 5 3)>
<Op (unstack 5 2)>
<Op (unstack 5 7)>
<Op (unstack 5 4)>
<Op (unstack 5 1)>
<Op (unstack 5 8)>
<Op (unstack 5 9)>
<Op (unstack 5 6)>
<Op (unstack 3 5)>
<Op (unstack 3 3)>
<Op (unstack 3 2)>
<Op (unstack 3 7)>
<Op (unstack 3 4)>
<Op (unstack 3 1)>
<Op (unstack 3 8)>
<Op (unstack 3 9)>
<Op (unstack 3 6)>
<Op (unstack 2 5)>
<Op (unstack 2 3)>
<Op (unstack 2 2)>
<Op (unstack 2 7)>
<Op (unstack 2 4)>
<Op (unstack 2 1)>
<Op (unstack 2 8)>
<Op (unstack 2 9)>
<Op (unstack 2 6)>
<Op (unstack 7 5)>
<Op (unstack 7 3)>
<Op (unstack 7 2)>
<Op (unstack 7 7)>
<Op (unstack 7 4)>
<Op (unstack 7 1)>
<Op (unstack 7 8)>
<Op (unstack 7 9)>
<Op (unstack 7 6)>
<Op (unstack 4 5)>
<Op (unstack 4 3)>
<Op (unstack 4 2)>
<Op (unstack 4 7)>
<Op (unstack 4 4)>
<Op (unstack 4 1)>
<Op (unstack 4 8)>
<Op (unstack 4 9)>
<Op (unstack 4 6)>
<Op (unstack 1 5)>
<Op (unstack 1 3)>
<Op (unstack 1 2)>
<Op (unstack 1 7)>
<Op (unstack 1 4)>
<Op (unstack 1 1)>
<Op (unstack 1 8)>
<Op (unstack 1 9)>
<Op (unstack 1 6)>
<Op (unstack 8 5)>
<Op (unstack 8 3)>
<Op (unstack 8 2)>
<Op (unstack 8 7)>
<Op (unstack 8 4)>
<Op (unstack 8 1)>
<Op (unstack 8 8)>
<Op (unstack 8 9)>
<Op (unstack 8 6)>
<Op (unstack 9 5)>
<Op (unstack 9 3)>
<Op (unstack 9 2)>
<Op (unstack 9 7)>
<Op (unstack 9 4)>
<Op (unstack 9 1)>
<Op (unstack 9 8)>
<Op (unstack 9 9)>
<Op (unstack 9 6)>
<Op (unstack 6 5)>
<Op (unstack 6 3)>
<Op (unstack 6 2)>
<Op (unstack 6 7)>
<Op (unstack 6 4)>
<Op (unstack 6 1)>
<Op (unstack 6 8)>
<Op (unstack 6 9)>
<Op (unstack 6 6)>) == 12

assert h_ff(Task bw-large-b
  Vars:  (on 8 7), (on 11 4), (on 3 7), (on-table 11), (on 8 1), (on 7 1), (on 10 10), (holding 4), (on 5 7), (clear 6), (on 11 1), (on 2 11), (on-table 8), (on 5 4), (on 5 11), (on 4 1), (on-table 4), (on 7 11), (on 2 3), (on 10 8), (clear 8), (on 3 4), (on 4 8), (on 2 2), (on 5 1), (on 8 3), (on 6 4), (on 1 8), (on 4 10), (on 4 5), (on 6 8), (on 2 5), (on 5 6), (on 9 1), (on 7 6), (on 11 11), (on 11 10), (on 3 9), (holding 10), (on 10 4), (clear 3), (on 1 1), (on 5 10), (on 4 7), (on 9 6), (on-table 7), (on 8 10), (on-table 9), (holding 3), (on 3 1), (on 3 11), (on 2 4), (on-table 10), (holding 6), (on 3 2), (on 8 5), (on 1 10), (holding 8), (on 10 6), (on 9 2), (on-table 5), (on 7 2), (on 3 10), (on 10 7), (on 8 9), (on 6 2), (arm-empty), (on 1 5), (on 4 4), (clear 9), (on 9 5), (on 6 9), (on 3 8), (on 6 10), (holding 11), (on 6 6), (on 5 2), (on 2 8), (on 7 3), (on 11 7), (on 2 7), (on 4 11), (on 9 7), (on 6 5), (on 11 5), (on 11 9), (on 10 2), (on 5 3), (clear 5), (on 3 6), (clear 10), (on 8 11), (on 11 6), (on 10 3), (on 1 4), (holding 2), (holding 5), (on 6 11), (on 4 6), (on 10 11), (on 2 6), (on 10 9), (on 7 9), (clear 7), (on-table 1), (clear 2), (on 3 3), (on 2 1), (on 11 8), (on 9 11), (on 8 2), (on 9 8), (on 10 5), (on 8 6), (on 5 9), (on 7 7), (on 7 8), (on 9 9), (on 8 4), (on 1 9), (on 9 3), (clear 11), (on 7 10), (on 11 2), (clear 4), (holding 9), (on 2 10), (on 8 8), (on-table 2), (on 1 7), (on 4 3), (on 4 9), (on 3 5), (on 10 1), (on 1 6), (on 5 8), (on 6 7), (on 1 3), (on 2 9), (on 9 10), (on 4 2), (clear 1), (on 11 3), (on-table 6), (on 9 4), (on 6 1), (holding 7), (on 5 5), (on 7 4), (on 7 5), (on-table 3), (holding 1), (on 1 2), (on 6 3), (on 1 11)
  Init:  frozenset({"(on 8 7)", "(on-table 4)", "(arm-empty)", "(on 7 6)", "(on 3 2)", "(clear 9)", "(on 2 1)", "(on-table 1)", "(on 11 10)", "(on 9 8)", "(clear 11)", "(on 10 5)", "(clear 3)", "(on 5 4)", "(on-table 6)"})
  Goals: frozenset({"(on-table 4)", "(on 3 11)", "(on 2 3)", "(on 11 7)", "(on 1 5)", "(on 7 6)", "(on-table 10)", "(on-table 6)", "(clear 8)", "(clear 2)", "(on 5 10)", "(clear 1)", "(on 9 4)", "(on 8 9)"})
  Ops:   <Op (pick-up 10)>
<Op (pick-up 5)>
<Op (pick-up 3)>
<Op (pick-up 2)>
<Op (pick-up 7)>
<Op (pick-up 4)>
<Op (pick-up 1)>
<Op (pick-up 8)>
<Op (pick-up 9)>
<Op (pick-up 11)>
<Op (pick-up 6)>
<Op (put-down 10)>
<Op (put-down 5)>
<Op (put-down 3)>
<Op (put-down 2)>
<Op (put-down 7)>
<Op (put-down 4)>
<Op (put-down 1)>
<Op (put-down 8)>
<Op (put-down 9)>
<Op (put-down 11)>
<Op (put-down 6)>
<Op (stack 10 10)>
<Op (stack 10 5)>
<Op (stack 10 3)>
<Op (stack 10 2)>
<Op (stack 10 7)>
<Op (stack 10 4)>
<Op (stack 10 1)>
<Op (stack 10 8)>
<Op (stack 10 9)>
<Op (stack 10 11)>
<Op (stack 10 6)>
<Op (stack 5 10)>
<Op (stack 5 5)>
<Op (stack 5 3)>
<Op (stack 5 2)>
<Op (stack 5 7)>
<Op (stack 5 4)>
<Op (stack 5 1)>
<Op (stack 5 8)>
<Op (stack 5 9)>
<Op (stack 5 11)>
<Op (stack 5 6)>
<Op (stack 3 10)>
<Op (stack 3 5)>
<Op (stack 3 3)>
<Op (stack 3 2)>
<Op (stack 3 7)>
<Op (stack 3 4)>
<Op (stack 3 1)>
<Op (stack 3 8)>
<Op (stack 3 9)>
<Op (stack 3 11)>
<Op (stack 3 6)>
<Op (stack 2 10)>
<Op (stack 2 5)>
<Op (stack 2 3)>
<Op (stack 2 2)>
<Op (stack 2 7)>
<Op (stack 2 4)>
<Op (stack 2 1)>
<Op (stack 2 8)>
<Op (stack 2 9)>
<Op (stack 2 11)>
<Op (stack 2 6)>
<Op (stack 7 10)>
<Op (stack 7 5)>
<Op (stack 7 3)>
<Op (stack 7 2)>
<Op (stack 7 7)>
<Op (stack 7 4)>
<Op (stack 7 1)>
<Op (stack 7 8)>
<Op (stack 7 9)>
<Op (stack 7 11)>
<Op (stack 7 6)>
<Op (stack 4 10)>
<Op (stack 4 5)>
<Op (stack 4 3)>
<Op (stack 4 2)>
<Op (stack 4 7)>
<Op (stack 4 4)>
<Op (stack 4 1)>
<Op (stack 4 8)>
<Op (stack 4 9)>
<Op (stack 4 11)>
<Op (stack 4 6)>
<Op (stack 1 10)>
<Op (stack 1 5)>
<Op (stack 1 3)>
<Op (stack 1 2)>
<Op (stack 1 7)>
<Op (stack 1 4)>
<Op (stack 1 1)>
<Op (stack 1 8)>
<Op (stack 1 9)>
<Op (stack 1 11)>
<Op (stack 1 6)>
<Op (stack 8 10)>
<Op (stack 8 5)>
<Op (stack 8 3)>
<Op (stack 8 2)>
<Op (stack 8 7)>
<Op (stack 8 4)>
<Op (stack 8 1)>
<Op (stack 8 8)>
<Op (stack 8 9)>
<Op (stack 8 11)>
<Op (stack 8 6)>
<Op (stack 9 10)>
<Op (stack 9 5)>
<Op (stack 9 3)>
<Op (stack 9 2)>
<Op (stack 9 7)>
<Op (stack 9 4)>
<Op (stack 9 1)>
<Op (stack 9 8)>
<Op (stack 9 9)>
<Op (stack 9 11)>
<Op (stack 9 6)>
<Op (stack 11 10)>
<Op (stack 11 5)>
<Op (stack 11 3)>
<Op (stack 11 2)>
<Op (stack 11 7)>
<Op (stack 11 4)>
<Op (stack 11 1)>
<Op (stack 11 8)>
<Op (stack 11 9)>
<Op (stack 11 11)>
<Op (stack 11 6)>
<Op (stack 6 10)>
<Op (stack 6 5)>
<Op (stack 6 3)>
<Op (stack 6 2)>
<Op (stack 6 7)>
<Op (stack 6 4)>
<Op (stack 6 1)>
<Op (stack 6 8)>
<Op (stack 6 9)>
<Op (stack 6 11)>
<Op (stack 6 6)>
<Op (unstack 10 10)>
<Op (unstack 10 5)>
<Op (unstack 10 3)>
<Op (unstack 10 2)>
<Op (unstack 10 7)>
<Op (unstack 10 4)>
<Op (unstack 10 1)>
<Op (unstack 10 8)>
<Op (unstack 10 9)>
<Op (unstack 10 11)>
<Op (unstack 10 6)>
<Op (unstack 5 10)>
<Op (unstack 5 5)>
<Op (unstack 5 3)>
<Op (unstack 5 2)>
<Op (unstack 5 7)>
<Op (unstack 5 4)>
<Op (unstack 5 1)>
<Op (unstack 5 8)>
<Op (unstack 5 9)>
<Op (unstack 5 11)>
<Op (unstack 5 6)>
<Op (unstack 3 10)>
<Op (unstack 3 5)>
<Op (unstack 3 3)>
<Op (unstack 3 2)>
<Op (unstack 3 7)>
<Op (unstack 3 4)>
<Op (unstack 3 1)>
<Op (unstack 3 8)>
<Op (unstack 3 9)>
<Op (unstack 3 11)>
<Op (unstack 3 6)>
<Op (unstack 2 10)>
<Op (unstack 2 5)>
<Op (unstack 2 3)>
<Op (unstack 2 2)>
<Op (unstack 2 7)>
<Op (unstack 2 4)>
<Op (unstack 2 1)>
<Op (unstack 2 8)>
<Op (unstack 2 9)>
<Op (unstack 2 11)>
<Op (unstack 2 6)>
<Op (unstack 7 10)>
<Op (unstack 7 5)>
<Op (unstack 7 3)>
<Op (unstack 7 2)>
<Op (unstack 7 7)>
<Op (unstack 7 4)>
<Op (unstack 7 1)>
<Op (unstack 7 8)>
<Op (unstack 7 9)>
<Op (unstack 7 11)>
<Op (unstack 7 6)>
<Op (unstack 4 10)>
<Op (unstack 4 5)>
<Op (unstack 4 3)>
<Op (unstack 4 2)>
<Op (unstack 4 7)>
<Op (unstack 4 4)>
<Op (unstack 4 1)>
<Op (unstack 4 8)>
<Op (unstack 4 9)>
<Op (unstack 4 11)>
<Op (unstack 4 6)>
<Op (unstack 1 10)>
<Op (unstack 1 5)>
<Op (unstack 1 3)>
<Op (unstack 1 2)>
<Op (unstack 1 7)>
<Op (unstack 1 4)>
<Op (unstack 1 1)>
<Op (unstack 1 8)>
<Op (unstack 1 9)>
<Op (unstack 1 11)>
<Op (unstack 1 6)>
<Op (unstack 8 10)>
<Op (unstack 8 5)>
<Op (unstack 8 3)>
<Op (unstack 8 2)>
<Op (unstack 8 7)>
<Op (unstack 8 4)>
<Op (unstack 8 1)>
<Op (unstack 8 8)>
<Op (unstack 8 9)>
<Op (unstack 8 11)>
<Op (unstack 8 6)>
<Op (unstack 9 10)>
<Op (unstack 9 5)>
<Op (unstack 9 3)>
<Op (unstack 9 2)>
<Op (unstack 9 7)>
<Op (unstack 9 4)>
<Op (unstack 9 1)>
<Op (unstack 9 8)>
<Op (unstack 9 9)>
<Op (unstack 9 11)>
<Op (unstack 9 6)>
<Op (unstack 11 10)>
<Op (unstack 11 5)>
<Op (unstack 11 3)>
<Op (unstack 11 2)>
<Op (unstack 11 7)>
<Op (unstack 11 4)>
<Op (unstack 11 1)>
<Op (unstack 11 8)>
<Op (unstack 11 9)>
<Op (unstack 11 11)>
<Op (unstack 11 6)>
<Op (unstack 6 10)>
<Op (unstack 6 5)>
<Op (unstack 6 3)>
<Op (unstack 6 2)>
<Op (unstack 6 7)>
<Op (unstack 6 4)>
<Op (unstack 6 1)>
<Op (unstack 6 8)>
<Op (unstack 6 9)>
<Op (unstack 6 11)>
<Op (unstack 6 6)>) == 16
print('Tests passed.')